# Script para crear archivo .nt y .nq

## 1. Archivo prostatecancer.nt

In [56]:
import csv
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import OWL, RDF, RDFS, SKOS, XSD, DC, DCTERMS, VOID

# Abrir archivo CSV
input_file = csv.DictReader(open("prostate_cancer.csv"))

# Crear un objeto Grafo 
output_graph = Graph()

# URI base
base_uri = 'https://prostatecancer.es/data/'

# Propiedades comunes (FAIR: reusar vocabularios)

GO="http://www.ebi.ac.uk/QuickGO/term/GO:"
NCIT="https://ncit.nci.nih.gov/ncitbrowser/ConceptReport.jsp?dictionary=NCI_Thesaurus&ns=ncit&code="
REACTOME="https://reactome.org/content/detail/"
SIO="https://semanticscience.org/resource/SIO_"
SO="http://purl.obolibrary.org/obo/SO_"
Ncit="http://purl.obolibrary.org/obo/NCIT_"
OGG="http://purl.obolibrary.org/obo/OGG_"
RO="http://purl.obolibrary.org/obo/RO_"
TXPO="http://purl.obolibrary.org/obo/TXPO_"
Go="http://purl.obolibrary.org/obo/GO_"
ENSEMBL="https://grch37.ensembl.org/Homo_sapiens/Gene/Summary?g="
UNIPROT="https://www.uniprot.org/uniprotkb/"
FMA="http://purl.obolibrary.org/obo/FMA_"
MI="http://purl.obolibrary.org/obo/MI_"
pubmed="http://pubmed.ncbi.nlm.nih.gov/" 

# Recorrer cada linea y convertir a triples RDF

for row in input_file:
    # Gene
    gen_name=row['gene']    
    gen_uri = OGG + row['id_gene']
    # Prostate_cancer -> Predicted_genes
    predicted_gene=SIO+"000987"    
    
    if gen_uri != 'NA':
        # prostate_cancer->predicted_gene->gen
        output_graph.add( (URIRef(base_uri), URIRef(predicted_gene), URIRef(gen_uri)))
        
        # gen->rdfs:label->nombre
        output_graph.add( (URIRef(gen_uri), RDFS.label, Literal(gen_name, lang='en')) )
    
    #IDs
    identifier=SIO+"000115"
    
    ensembl=row['Ensembl']
    ensembl_uri=ENSEMBL+ensembl

    uniprot=row['Uniprot']
    uniprot_uri=UNIPROT+uniprot
    
    
    # gen->identifier->uri(ensembl/Uniprot)
    output_graph.add( (URIRef(gen_uri), URIRef(identifier), URIRef(ensembl_uri)) )
    output_graph.add( (URIRef(gen_uri), URIRef(identifier), URIRef(uniprot_uri)) )
    
    # uri(ensembl/Uniprot)->rdfs:label->id (ensembl/Uniprot)
    output_graph.add( (URIRef(ensembl_uri), RDFS.label, Literal(ensembl, lang='en')) )
    output_graph.add( (URIRef(uniprot_uri), RDFS.label, Literal(uniprot, lang='en')) )
    
    #Gene description
    
    description=row['Gene.description']
    description_uri=SIO+"000136"
    output_graph.add( (URIRef(gen_uri), URIRef(description_uri), Literal(description, lang='en')) )
    
    
    #Chromosome
    chromosome=row['chromosome']
    chromosome_uri=NCIT+row['chromosome_uri']
    is_contained_in=SIO+"000128"
    
    output_graph.add( (URIRef(gen_uri), URIRef(is_contained_in), URIRef(chromosome_uri)))
    output_graph.add( (URIRef(chromosome_uri), RDFS.label, Literal(chromosome, lang='en')) )
    
    #Position
    position=row['Position']
    genomic_position=Ncit+"C123886"
    output_graph.add( (URIRef(gen_uri), URIRef(genomic_position), Literal(position, lang='en')) )
    

    # Mutation and type mutation
    mutation = row['mutation']
    sequence_alteration=SO+"0001059"
    type_mutation = row['type_mutation']
    deletion=SO+"0000159"
    substitution=SO+"1000002"
    has=Ncit+"C101282"
    
    if type_mutation == "Substitution": 
        output_graph.add((URIRef(gen_uri),URIRef(substitution),Literal(mutation, lang='en')) )
    if type_mutation == "Deletion":
        output_graph.add((URIRef(gen_uri),URIRef(deletion),Literal(mutation, lang='en')) )
        
    # Biological process
    biological_process=Ncit+"C17828"
    process=row['biological_process']
    id_function=row['id_function']
    process_uri=GO+id_function
    if id_function != 'NA':
        output_graph.add( (URIRef(gen_uri), URIRef(biological_process), URIRef(process_uri)))
        output_graph.add( (URIRef(process_uri), RDFS.label, Literal(process, lang='en')) )
    
    # Cellular component
    cellular_component=Go+"0005575"
    component=row['cellullar.component']
    id_component=row['id_component']
    component_uri=GO+id_component
    if id_component != 'NA':
        output_graph.add( (URIRef(gen_uri), URIRef(cellular_component), URIRef(component_uri)))
        output_graph.add( (URIRef(component_uri), RDFS.label, Literal(component, lang='en')) )
    
    # Molecular function
    molecular_function=Go+"0003674"
    molecular=row['molecular_function']
    id_molecular=row['id_molecular']
    molecular_uri=GO+id_molecular
    if id_molecular != 'NA':
        output_graph.add( (URIRef(gen_uri), URIRef(molecular_function), URIRef(molecular_uri)))
        output_graph.add( (URIRef(molecular_uri), RDFS.label, Literal(molecular, lang='en')) )
    
    # REACTOME
    participes=row['participes_in']
    candidate_of=Ncit+"C138079"
    component_of=TXPO+"0000039"
    member_of=RO+"0002350"
    input_of=RO+"0002352"
    output_of=RO+"0002353"
    #participes_uri=row['uris_participes_in']
    participes_label=row['reactome_label']
    reactome_uri=REACTOME+row['reactome_id']
    
    if participes == "candidate_of":
        output_graph.add((URIRef(gen_uri), URIRef(candidate_of), URIRef(reactome_uri)))
        output_graph.add((URIRef(reactome_uri), RDFS.label, Literal(participes_label,lang='en')))
    if participes== "component_of":
        output_graph.add((URIRef(gen_uri), URIRef(component_of), URIRef(reactome_uri)))
        output_graph.add((URIRef(reactome_uri), RDFS.label, Literal(participes_label,lang='en')))   
    if participes == "member_of":
        output_graph.add((URIRef(gen_uri), URIRef(member_of), URIRef(reactome_uri)))
        output_graph.add((URIRef(reactome_uri), RDFS.label, Literal(participes_label,lang='en')))
    if participes == "input_of":
        output_graph.add((URIRef(gen_uri), URIRef(input_of), URIRef(reactome_uri)))
        output_graph.add((URIRef(reactome_uri), RDFS.label, Literal(participes_label,lang='en')))
    if participes == "output_of":
        output_graph.add((URIRef(gen_uri), URIRef(output_of), URIRef(reactome_uri)))
        output_graph.add((URIRef(reactome_uri), RDFS.label, Literal(participes_label,lang='en')))
    
    # Compartment of reactome
    compartment_cell=FMA+"223408"
    reactome_compartment=row['go_compartment']
    go_compartment_uri=row['go_compartment_uri']
    reactome_compartment_uri=GO+"000"+go_compartment_uri
    if go_compartment_uri != 'NA': 
        output_graph.add((URIRef(reactome_uri), URIRef(compartment_cell), URIRef(reactome_compartment_uri)))
        output_graph.add((URIRef(reactome_compartment_uri), RDFS.label, Literal(reactome_compartment,lang='en')))
    
    # Article
    article=NCIT+"C47902"
    article_uri=pubmed+row['id_article']
    article_label=row['name_article']
    article_year=row['year_article']
    publication_year=MI+"0886"
    
    if gen_name != 'KAT6A':   
        output_graph.add((URIRef(gen_uri), URIRef(article), URIRef(article_uri)))
        output_graph.add((URIRef(article_uri), RDFS.label, Literal(article_label,lang='en')))
        output_graph.add((URIRef(article_uri), URIRef(publication_year), Literal(article_year,lang='en')))

output_graph.serialize(destination='prostatecancer.nt', format='nt')

## 2. Archivo prostatecancer.nq

In [57]:
import os

nombre_archivo_original = "prostatecancer.nt"  # Reemplaza "archivo.txt" por la ruta y nombre de tu archivo original
nombre_archivo_nuevo = "prostatecancer.nq"  # Reemplaza "nuevo_archivo.txt" por el nombre que deseas para el archivo modificado

# Abrir el archivo original en modo lectura
with open(nombre_archivo_original, "r") as archivo_original:
    lineas = archivo_original.readlines()  # Leer todas las líneas del archivo original

# Realizar los reemplazos en cada línea
nuevo_contenido = []
for linea in lineas:
    elementos = linea.rstrip("\n").split()  # Dividir la línea en elementos separados por espacios
    if len(elementos) > 1:  # Verificar que haya más de un elemento en la línea
        nuevos_elementos = elementos[:-1]  # Eliminar el último elemento de la línea
        nuevos_elementos.append("<https://prostatecancer.es/graph/genes_predicted> .")  # Añadir el nuevo contenido al final de la línea
        nueva_linea = " ".join(nuevos_elementos) + "\n"  # Unir los elementos de la línea con espacios y agregar el carácter de salto de línea
        nuevo_contenido.append(nueva_linea)

# Abrir el archivo nuevo en modo escritura para guardar los cambios
with open(nombre_archivo_nuevo, "w") as archivo_nuevo:
    archivo_nuevo.writelines(nuevo_contenido)

print("Reemplazos realizados con éxito.")

# Renombrar el archivo nuevo con el formato deseado
nuevo_nombre_con_formato = "prostatecancer.nq" # Reemplaza "nuevo_archivo.txt" por el nuevo nombre y formato deseado, por ejemplo: "nuevo_archivo.csv"
os.rename(nombre_archivo_nuevo, nuevo_nombre_con_formato)

print("Archivo renombrado con éxito.")


Reemplazos realizados con éxito.
Archivo renombrado con éxito.
